# Multiplicação de Matrizes (MM), no PCAD

## Instanciação da infraestrutura

O professor fará a demonstração de um escalonador e trabalhadores no PCAD (https://gppd-hpc.inf.ufrgs.br/), utilizando pelo menos duas máquinas (poderemos eventualmente usar mais máquinas em função da disponibilidade dos nós computacionais):
- `cei1`: o escalonador juntamente com trabalhadores na máquinas
- `cei2`: somente trabalhadores

O professor vai demonstrar a instanciação manual dessa infraestrutura de computação utilizando conectando diretamente nas máquinas via `ssh`, instanciando primeiro o escalonador com `dask-scheduler`, obtendo o seu *endereço:porta* e depois instanciando tantos quantos `dask-worker` forem necessários, informando para cada um destes o endereço do escalonador.

## Verificação da disponibilidade da infraestrutura

Após a instanciação da infraestrutura, poderemos então acessar o endereço do `dashboard` do dask onde constarão dados da infraestrutura que foi instanciada.
- http://gppd-hpc.inf.ufrgs.br:8787/status

## Experimento de MM

No código abaixo vamos conectar nosso cliente local no endereço do escalonador.

Vamos investigar a percepção do cliente da infraestrutura, através da saída.

In [1]:
!python3 --version

Python 3.10.6


In [2]:
from dask.distributed import Client
client = Client("tcp://gppd-hpc.inf.ufrgs.br:8786")
client

/home/s113650157/Dev/miniconda3/envs/cd009/lib/python3.10/site-packages/distributed/client.py:1309: VersionMismatchWarning: Mismatched versions found

+---------+----------------+----------------+----------------+
| Package | Client         | Scheduler      | Workers        |
+---------+----------------+----------------+----------------+
| python  | 3.10.6.final.0 | 3.10.7.final.0 | 3.10.7.final.0 |
+---------+----------------+----------------+----------------+
  warnings.warn(version_module.VersionMismatchWarning(msg[0]["warning"]))


<Client: 'tcp://143.54.48.109:8786' processes=8 threads=8, memory=15.51 GiB>

Vamos agora instanciar as duas matrizes que queremos multiplicar, e visualizá-las.

In [9]:
import dask.array as da
A = da.random.random((48000,1200), chunks=(6000, -1))
A

dask.array<random_sample, shape=(48000, 1200), dtype=float64, chunksize=(6000, 1200), chunktype=numpy.ndarray>

In [10]:
B = da.random.random((1200,48000), chunks=(-1, 6000))
B

dask.array<random_sample, shape=(1200, 48000), dtype=float64, chunksize=(1200, 6000), chunktype=numpy.ndarray>

Vamos agora criar o grafo de tarefas da MM. Alguns questionamentos:
- Qual a capacidade de paralelismo observando o grafo?
- Existem dependências globais?

In [11]:
from numpy import save
C = da.matmul(A, B).mean()
C.dask

HighLevelGraph with 8 layers.
<dask.highlevelgraph.HighLevelGraph object at 0x7f5d1859d270>
 0. random_sample-7d3247a7f1cb2498e5a47b7795dc43ab
 1. random_sample-a4f35c98a6e0f6eb9e2f018db825de00
 2. matmul-ac653907e83e865ef8f7e46b734d6c33
 3. getitem-2380b8f29a758b133c6a17acd8087ae6
 4. mean_chunk-40f1176826727f35f3f626949601afe7
 5. mean_combine-partial-55477fe2cdaba7be6e1cac441094c0ac
 6. mean_combine-partial-c240811e551d10971ad0e8a7aa253e9f
 7. mean_agg-aggregate-a39479b43e6beb546dddc6ec6e06213b

Agora vamos solicitar o cálculo com uma chamada para `compute()`.

In [12]:
from timeit import default_timer as timer
from dask.distributed import performance_report
with performance_report(filename="dask-report.html"):
  start = timer()
  Z = C.compute()
  end = timer()
  print(end - start)

302.74184443900594


In [ ]:
cluster = LocalCluster